# Birdpark Dataset Conversion

Data from Rüttimann et al. (2024)¹ ([paper](https://peerj.com/articles/20203), [zenodo](https://zenodo.org/records/13144875)), a multimodal dataset of zebra finch groups with synchronized video, microphone arrays, and backpack-mounted vibration transducer (accelerometers). 

The code belows how one can convert that existing dataset into the `Trials.nc` format. The sampling rate of the vibration transducer is very high (24kHz), therefore if the bottom plot loads very slowly, you may consider using the `Downsample` button in `I/0`.

---

¹ Rüttimann, L., Wang, Y., Rychen, J., Tomka, T., Hörster, H., & Hahnloser, R. H. R. (2025). Multimodal system for recording individual-level behaviors in songbird groups. PeerJ, 13, e20203. https://doi.org/10.7717/peerj.20203



<img src="assets/birdpark1.png" width="900">

Adapated from Rüttimann et al. (2024)¹ - Fig. 1A-B, Fig. 2C

<img src="assets/birdpark2.png" width="1200">

GUI screenshot

In [ ]:
import os 
import xarray as xr
import numpy as np
import h5py
import requests
import zipfile
from pathlib import Path
from audioio import write_audio
from ethograph import set_media_attrs, minimal_basics, get_project_root

project_root = Path.cwd().parent
os.chdir(project_root)

### Explore snippet

Example data from the birdpark dataset (a 7-second snippet from the `copExpBP08` recording) is available in:

- `data/examples/copExpBP08_trim.mp4` - Video file
- `data/examples/copExpBP08_trim.wav` - Audio file
- `data/examples/copExpBP08_trim.nc` - Accelerometer data and metadata

You can open these files directly from the GUI to explore the dataset.

### Download dataset

You can download the entire dataset from [here](https://zenodo.org/records/13144875) or use the code below. I only tested the `copExpBP08` recording. If problems arise, the ReadMe [here](https://zenodo.org/records/13144875) is very helpful.

In [8]:
data_folder = get_project_root() / "data" / "birdpark"



response = requests.get("https://zenodo.org/api/records/13144875")
data = response.json()

for file in data['files']:
    if file['checksum'] == "md5:32d1ae6049556c803f68b6d354c952ca":
        print(f"Checksum matches: {file['key']}")
        download_url = file['links']['self']
        
        output_path = data_folder / file['key']
        response = requests.get(download_url, stream=True)
        with open(output_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        
        print(f"Downloaded: {output_path}")
        
        # Unzip if it's a zip file
        if output_path.suffix == '.zip':
            print(f"Extracting {output_path.name}...")
            with zipfile.ZipFile(output_path, 'r') as zip_ref:
                zip_ref.extractall(data_folder)
            print(f"Extracted to: {data_folder}")
        
        break

Checksum matches: Data.zip
Downloaded: C:\Users\aksel\Documents\Code\EthoGraph\data\Data.zip
Extracting Data.zip...
Extracted to: C:\Users\aksel\Documents\Code\EthoGraph\data


In [17]:
fps = 47.6837158203125
audio_sr = 24414.0625 # audio and accelerometer sampling rate

h5_path = data_folder / "copExpBP08" / "BP_2021-05-25_08-12-51_655154_0380000.h5"
video_path = data_folder / "copExpBP08" / "BP_2021-05-25_08-12-51_655154_0380000.mp4"
audio_path = video_path.with_suffix('.wav')
nc_path = video_path.with_suffix('.nc')

In [13]:
# get file object
f1 = h5py.File(h5_path, 'r')

# read main data
radioSignals = f1['/radioSignals'][()] # accelerometer transmitter device signals (one row per channel)
daqSignals = f1['/daqSignals'][()] # microphone signals (one row per channel)

In [ ]:
# Create .wav file with microphone channels
write_audio(audio_path, daqSignals.T, audio_sr)

In [18]:
# Setup Trials.nc
time_coords = np.arange(radioSignals.shape[1]) / audio_sr

ds = xr.Dataset(
        data_vars={
            "vibration": xr.DataArray(
                radioSignals.T,
                dims=["time", "individuals"],
            ),
        },
        coords={
            "time": time_coords,
            "individuals": ["male (red radio)", "female (yellow radio)"], # Specific to copExpBP08
        },
        attrs={
            "fps": fps,
            "audio_sr": audio_sr
            
        }
    )

ds = set_media_attrs(
        ds,
        cameras=[video_path.name],
        mics=[audio_path.name])

dt = minimal_basics(ds)
dt.to_netcdf(nc_path)

Extracted type_vars_dict: {'individuals': array(['male (red radio)', 'female (yellow radio)'], dtype='<U21'), 'cameras': array(['cam1'], dtype='<U4'), 'mics': array(['mic1'], dtype='<U4'), 'features': ['vibration'], 'trial_conditions': []}
